Transcripcion del ejercicio:

# ***"PROYECTO: SERVICIO AUTOMOTRIZ AUTO MOTORS"***
Empresa colombiana con franquicias . Ofrece servicios y productos como cambio de
aceite, cambio de filtros, venta de cubiertas, llantas, entre otras.

# ***Descripción del proyecto:***

Contamos con datos internos de la empresa como ventas y empleados.
Requieren analizar el impacto de la pandemia para evaluar si ven der la empresa, si quiebra o si salir a buscar inversores para expansión hacia otros países.

# ***Objetivo:***

El cliente necesita contar con toda la información ya anal izada para tomar una decisión final.

# ***Indicadores básicos para empezar:***

Venta Total

Cantidad de Elementos vendidos

Ganancia Facturación Total de Servicios

Cantidad de Servicios Ofrecidos

Cantidad de Sedes o ubicaciones disponibles

Ventas totales por Sedes en cada Rubro o Familia y Servicios

Ventas realizadas por sus empleados

Ranking de Ventas / Servicios de Empleados


*+Agregar todos los indicadores que crean correspondientes*.

# ***TRABAJO PARA ANALISTAS DE DATOS***

Crear de manera visual gráficos, estadísticas y números finales para anali zar toda la información ofrecida en dos datasets.

Uno contiene información del año 2016 al 2020 de la empresa.
El otro contiene información relacionada a los empleados.

# ***A tener en cuenta:***
La decisión de la empresa tiene que verse reflejada tant o en la rentabilidad, como también en el manejo y cantidad de personal.

Para este proyecto debemos manejarnos en dólares estadunidense para los reportes (0,00023).
"

#Importamos todas las librerías que por ahora creemos necesitar (Y digo por ahora, porque puede ser que necesitemos mas):

In [ ]:
!pip install 'dtreeviz'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 2.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import time

#Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import dtreeviz.trees as dtreeviz

#modelos y métricas
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score,f1_score, precision_recall_curve, roc_curve
from sklearn.metrics import confusion_matrix, classification_report
#preprocesamiento
from sklearn.preprocessing import MinMaxScaler

#configuración warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#***Análisis exploratorio***

Veamos y evaluemos los dos datasets:

In [ ]:
# Leer el archivo CSV y omitir las líneas con 12 ";"
lines = []
with open('datasets/Ventas.csv', 'r') as f:
    for line in f:
        if line.strip() != ';' * 12:
            lines.append(line)

# Crear un archivo sin las líneas no deseadas
with open('auxiliar_ventas.csv', 'w') as temp_file:
    temp_file.writelines(lines)

# Leer el archivo temporal con Pandas
df_ventas = pd.read_csv('auxiliar_ventas.csv', sep=';')
ds_trabajo_ventas = df_ventas.copy()

<ipython-input-3-d36deea80dcd>:13: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.


In [ ]:
ds_empleados = pd.read_csv('./datasets/Empleados.csv')
ds_trabajo_empleados = ds_empleados.copy()

El dataset de la lista de registros de ventas:

In [ ]:
df_ventas.head()

,IdCliente,NombreCliente,Fecha,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area
0,C1070956433,PINTO JOHN,1/9/2016,24,LL-2R-0038,PILOT STREET 110/80-17,101,LLANTA,1,129310,Puente aranda,Calle 13,35
1,C1032372566,CESAR CRUZ,1/9/2016,14,LL-TCA-0133,TOURING 175/70R13,101,LLANTA,2,181034,Suba,Suba,32
2,C2865810,PINEDA JAIME,1/9/2016,12,LL-TCA-0019,ENERGY XM2 185/65R14,101,LLANTA,1,193095,Usaquen,Santa ana,33
3,C11210285,VILLALBA RODRIGUEZ FABIO,1/9/2016,24,LL-TCA-0009,ENERGY XM2 165/65R13,101,LLANTA,2,262069,Puente aranda,Calle 13,35
4,C80755314,MOLINA RONALD,1/9/2016,14,LL-TCA-0132,NO UTILIZAR,101,LLANTA,4,362069,Suba,Suba,32


El dataset de la lista de registros de los empleados:

In [ ]:
ds_empleados.head()

,Id_Empleado;Nombre y Apellido
0,1;Juan Pastran
1,2;Esteban Damico
2,3;Julian Quevedo
3,4;Oscar Rosseto
4,5;Valentin Quintero


Listar la cantidad de filas y columnas que tienen ambos datasets

In [ ]:
ds_trabajo_ventas.shape

(127255, 13)

In [ ]:
ds_trabajo_empleados.shape

(68, 1)

Partiremos en dos columnas el dataset de empleados porque no esta bueno tener el id y el nombre y apellido en una misma columna:

In [ ]:
# Dividir la columna en dos nuevas columnas
ds_trabajo_empleados[['id_empleado', 'nombre y apellido']] = ds_trabajo_empleados['Id_Empleado;Nombre y Apellido'].str.split(';', 1, expand=True)
# Eliminar la columna deseada
ds_trabajo_empleados = ds_trabajo_empleados.drop(columns=['Id_Empleado;Nombre y Apellido'])

In [ ]:
ds_trabajo_empleados.head()

,id_empleado,nombre y apellido
0,1,Juan Pastran
1,2,Esteban Damico
2,3,Julian Quevedo
3,4,Oscar Rosseto
4,5,Valentin Quintero


In [ ]:
ds_trabajo_empleados.shape

(68, 2)

El dataset de ventas contiene un total de 127255 filas y 13 columnas. Mientras que el dataset de empleados contiene un total de 68 filas y 2 columnas.

In [ ]:
ds_trabajo_ventas.columns.tolist()

['IdCliente',
 'NombreCliente',
 'Fecha',
 'Empleado',
 'Referencia',
 'Descripcion',
 'CodigoFamilia',
 'Familia',
 'Cantidad',
 'Ventas',
 'Localidad',
 'Sede',
 'Area']

In [ ]:
ds_trabajo_empleados.columns.tolist()

['id_empleado', 'nombre y apellido']

**Descripcion de los campos del dataset de empleados:**



*   id_empleado: Contiene el numero de id de cada empleado.
*   Nombre y Apellido: Nombre y apellido de cada uno de los empleados.






**Descripcion de los campos del dataset de ventas:**


*   IdCliente: Código de cada cliente.

*   NombreCliente: Nombre y apellido de cada cliente.

*   Fecha: Fecha en que se llevó a cabo la venta.

* Empleado: Número entero que representa al empleado que atendió la venta. Es posible que este número se corresponda con una tabla externa que relacione los números con los nombres de los empleados.

* Referencia: Un código de referencia para la venta.

* Descripcion: Descripción del producto o servicio vendido.

* CodigoFamilia: Código que indica la categoría o familia del producto.

* Familia: Nombre de la categoría o familia del producto.

* Cantidad: Cantidad de productos o servicios vendidos en esta transacción.

* Ventas: Valor total en la moneda local de la venta.

* Localidad: Localidad o área geográfica relacionada con la venta.


*   Elemento de lista


*   Elemento de lista


*   Elemento de lista



